# Building a house (Harvard Business Review, 1963)

Several tasks must be completed in order to build our house. Each task has a duration, tasks may be worked on simultaneously, but there is also a precedence relation. Some tasks can only be started once other tasks have been completed. The following table shows each task, it's duration, and the tasks that must be completed before it starts. How fast can the house be built?
![alt text](https://hbr.org/resources/images/article_assets/hbr/6309/63508_A.gif)

## Problem Data

In [2]:
#This array stores the task names (:a, :b, ..., :x)
tasks = []
for i = 'a':'x'
    #Can convert back to string using string(sym), i.e. string(:hello) returns "hello"
    push!(tasks, Symbol(i))
end

#Dictionary to store the project durations
dur = [0, 4, 2, 4, 6, 1, 2, 3, 2, 4, 10, 3, 1, 2, 3, 2, 1, 1, 2, 3, 1, 2, 5, 0]
duration = Dict(zip(tasks,dur))

#Dictionary to store the projects that a given project depends on (ancestors)
pre = ( [], [:a], [:b], [:c], [:d], [:c], [:f], [:f], [:d], [:d,:g], [:i,:j,:h], [:k],
    [:l], [:l], [:l], [:e], [:p], [:c], [:o,:t], [:m,:n], [:t], [:q,:r], [:v], [:s,:u,:w])
pred = Dict(zip(tasks,pre));

In [3]:
using JuMP, HiGHS
m = Model(HiGHS.Optimizer)

@variable(m, start[tasks])


#for i in tasks
#    for j in pred[i]
        #Task i must start later than the start of task j plus the duration of task j
#        @constraint(m, start[i] >= start[j] + duration[j])
#    end
#end
#One-line implementation of the constraints:
@constraint(m, link[i in tasks, j in pred[i]], start[i] >= start[j] + duration[j])

#Initial condition
@constraint(m, start[:a] == 0)
# total duration is start time of last task + duration of last task.
@objective(m, Min, start[:x] + duration[:x]);

print(m)

In [4]:
optimize!(m)

Presolving model
30 rows, 22 cols, 59 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-32); columns 0(-24); elements 0(-63) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  3.4000000000e+01
HiGHS run time      :          0.00


In [5]:
println("Start times: ", value.(start))
println("Minimum duration: ", objective_value(m))

Start times: 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, Any[:a, :b, :c, :d, :e, :f, :g, :h, :i, :j  …  :o, :p, :q, :r, :s, :t, :u, :v, :w, :x]
And data, a 24-element Vector{Float64}:
 -0.0
 -0.0
  4.0
  6.0
 10.0
  6.0
  7.0
  7.0
 10.0
 10.0
 14.0
 24.0
 27.0
 27.0
 27.0
 16.0
 18.0
  6.0
 32.0
 29.0
 32.0
 19.0
 21.0
 34.0
Minimum duration: 34.0
